In [1]:
from sklearn.svm import SVR
from minepy import MINE
import pandas as pd
import numpy as np

In [41]:
# load the csv data file
df = pd.read_csv("./data/nci60.csv")
# X = np.array(df.iloc[:,2:])
# y = np.array(df.iloc[:,1])
X = df.iloc[:,2:]
y = df.iloc[:,1]
type(X)

pandas.core.frame.DataFrame

In [84]:
# Define the rfe-svm function
def rfe_svm_mic(X, y, num_feature_to_selected):
    import time
    clf = SVR()
    model = clf.set_params(kernel='linear')
    X_new = X
    record = {}
    del_ = X.shape[1]-num_feature_to_selected
    start_time = time.time()
    for i in range(del_):
        # get the column name
        names = list(X_new)
        # fit the svm and get the weights of features
        fit = model.fit(X_new, y)
        # compute the mic scores
        mine = MINE()
        mic_scores = []
        for i in range(X_new.shape[1]):
            mine.compute_score(X_new.iloc[:,i], y)
            m = mine.mic()
            mic_scores.append(m)
        sum_cor = {}
        # sum_cor = (1-0.3)*svm_coef + 0.3 * mic_scores
        for i in range(X_new.shape[1]):
            sum_cor[names[i]] = 0.7 * fit.coef_[0][i] + 0.3 * mic_scores[i]
        # get the column name with the samllest value
        min_colname = min(sum_cor, key=sum_cor.get)
        # print(min_colname)
        # get the column number with the samllest value
        min_colnum = X.columns.get_loc(min_colname)
        # print(min_colnum)
        # print(sum_cor[min_colname])
        # remove the feature with the smallest weight
        X_new = X_new.drop(min_colname, axis=1)
        # record the index and value of the deleted feature
        # record.update({min_colname:sum_cor[min_colname]})
        record[min_colname] = sum_cor[min_colname]
    elapsed_time = time.time() - start_time
    return X_new, record, elapsed_time

In [85]:
# Do the feature selecltion and get the selected features
X_new, record, elapsed_time = rfe_svm_mic(X, y, 50)
#for key,value in record.items():
#    print(key,':',value)
len(record)
elapsed_time

21062.710179567337

In [89]:
X_new.shape[1]
len(record)
X_new.to_csv("rfe_svm_mic_out.csv")

In [116]:
X_new.insert(0, "IC50", y)
X_new.to_csv("rfe_svm_mic_out.csv")

In [101]:
# Do the cross-validation
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=10)
clf = SVR()
# model = clf.set_params(kernel='linear')
score = cross_val_score(clf, X_new, y, cv=k_fold, n_jobs=-1)
np.mean(score)

0.097509270691458871

In [111]:
#from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
k_fold = KFold(n_splits=2)
#rf = RandomForestRegressor()
lf= LinearRegression()
# model = clf.set_params(kernel='linear')
score = cross_val_score(lf, X_new, y, cv=k_fold, n_jobs=-1)
np.mean(score)

0.011140298089407641

In [23]:
clf = SVR()
model = clf.set_params(kernel='linear')
fit = model.fit(X, y)
fit.coef_.shape
fit.coef_[0][10663]

-0.0088323808322437469

In [22]:
mine = MINE()
mic_scores = []
for i in range(X.shape[1]):
    mine.compute_score(X.iloc[:,i], y)
    m = mine.mic()
    mic_scores.append(m)
len(mic_scores)
mic_scores[10663]

0.20484786985384534

In [28]:
# sum_cor = (1-0.3)*svm_coef + 0.3 * mic_scores
sum_cor = {}
names = list(X)
for i in range(X.shape[1]):
   sum_cor[names[i]] = 0.7 * fit.coef_[0][i] + 0.3 * mic_scores[i]


In [51]:
sum_cor['A4GALT']

0.063553530139019584

In [32]:
0.7 * fit.coef_[0][0] + 0.3 * mic_scores[0]

0.065592459545082221

In [33]:
min(sum_cor, key=sum_cor.get)

'GOT2'

In [42]:
X.columns.get_loc("GOT2")

3779